In [1]:
!pip install tensorflow onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 40.3 MB/s eta 0:00:00a 0:00:01
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 104.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 88.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 100.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 78.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1

In [21]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 34.1 MB/s eta 0:00:00a 0:00:01
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)


In [1]:
import numpy as np
import onnx
import os
import glob
from google.protobuf.json_format import MessageToDict
from onnx.external_data_helper import load_external_data_for_model

from onnx import numpy_helper
import onnxruntime as ort

model = onnx.load('ex/model.onnx')
for _input in model.graph.input:
    m_dict = MessageToDict(_input)
    dim_info = m_dict.get("type").get("tensorType").get("shape").get("dim")  # ugly but we have to live with this when using dict
    input_shape = [d.get("dimValue") for d in dim_info]  # [1, 3, 800, 800]
    print(input_shape)
    print(m_dict)  # 1
test_data_dir = 'ex/test_data_set_0'


session = ort.InferenceSession('ex/model.onnx')
input_info = session.get_inputs()[0]
input_shape = input_info.shape
print(input_shape)


# Load inputs
inputs = []
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    print(input_file)
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
        
    inputs.append(numpy_helper.to_array(tensor))


# Run the model on the backend
inputs = np.concatenate(inputs, axis=0)
print(inputs.shape)
prt_session = ort.InferenceSession('ex/model.onnx')
outputs = prt_session.run(None, {'input': inputs})

print(outputs)


[None, '1', '128', '128', '128']
{'name': 'input', 'type': {'tensorType': {'elemType': 1, 'shape': {'dim': [{'dimParam': 'batch_size'}, {'dimValue': '1'}, {'dimValue': '128'}, {'dimValue': '128'}, {'dimValue': '128'}]}}}}
['batch_size', 1, 128, 128, 128]
ex/test_data_set_0/input_0.pb
(8, 1, 128, 128, 128)
[array([[[[[  8.114596  ,   8.51679   ,   8.714801  , ...,
             9.044031  ,   8.497932  ,   7.952525  ],
          [  8.701483  ,   9.624597  ,  10.051586  , ...,
            10.540443  ,   9.689313  ,   8.491743  ],
          [  9.104427  ,  10.165402  ,  10.728822  , ...,
            11.407544  ,  10.354088  ,   8.887017  ],
          ...,
          [  9.018058  ,   9.925455  ,  10.493921  , ...,
            12.316056  ,  11.145791  ,   9.380472  ],
          [  8.889473  ,   9.691139  ,  10.196373  , ...,
            11.941708  ,  10.970497  ,   9.315464  ],
          [  8.489617  ,   8.936736  ,   9.2178755 , ...,
            10.434354  ,   9.762529  ,   8.649029  ]],

   